In [6]:
#API KEY Initialization
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ")
langsmith_api_key = os.getenv("LANGSMITH")

In [8]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = langsmith_api_key
os.environ["LANGCHAIN_PROJECT"] = "LangGraph"

In [9]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key, model_name = "gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E2BFC6C980>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E2BFC6ECF0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

## Start Building ChatBot using LangGraph

In [10]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [12]:
class State(TypedDict):
    '''
    Messages have the type "list", the "add_messages" function
    in the annotation defines how this state key should be updated.
    [In this case, it appends messages to the list, rather than overwriting them]
    '''
    messages:Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [13]:
graph_builder

In [14]:
def chatbot(state:State):
    return {"messages":llm.invoke(state['messages'])}

In [15]:
graph_builder.add_node("chatbot", chatbot)